In [109]:
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_auc_score,roc_curve

In [110]:
data = pd.read_csv('sales-data.csv',parse_dates=[0])
data.drop(['Mobile',"EMAIL"],axis=1,inplace=True)
data.dropna(inplace=True)
data.reset_index(inplace=True,drop=True)
data.head(2)

,Created,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,2018-11-13 18:51:00,9.0,Live Chat-Google Organic,Sales-Agent-3,Bangalore,Mode-1,Potential
1,2018-11-13 18:48:00,19.0,Call,Sales-Agent-4,Other Locations,Mode-5,Potential


In [111]:
Counter(data.Location)

Counter({'AUSTRALIA': 25,
         'Bangalore': 2079,
         'Chennai': 907,
         'Delhi': 470,
         'EUROPE': 3,
         'Howrah': 1,
         'Hyderabad': 527,
         'Kolkata': 55,
         'Malaysia': 4,
         'Mumbai': 399,
         'Other Locations': 2481,
         'Pune': 139,
         'Singapore': 17,
         'Trivandrum': 58,
         'UAE': 77,
         'UK': 41,
         'USA': 45})

In [112]:
#data = data[data.Location=='Bangalore']
data = data[(data.Location=='Bangalore') ]

In [113]:
data.loc[(data.Status=='CONVERTED'),'Status']='GOOD'
data.loc[(data.Status=='In Progress Positive'),'Status']='GOOD'
data.loc[(data.Status=='converted'),'Status']='GOOD'
data.loc[(data.Status=='Potential'),'Status']='GOOD'
data.loc[(data.Status=='Long Term'),'Status']='GOOD'


data.loc[(data.Status=='Open'),'Status']='BAD'
data.loc[(data.Status=='In Progress Negative'),'Status']='BAD'
data.loc[(data.Status=='Junk Lead'),'Status']='BAD'
data.loc[(data.Status=='Just Enquiry'),'Status']='BAD'
data.loc[(data.Status=='LOST'),'Status']='BAD'
data.loc[(data.Status=='Not Responding'),'Status']='BAD'

In [114]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
data.Product_ID = enc.fit_transform(data.Product_ID)
data.Source = enc.fit_transform(data.Source)
data.Sales_Agent = enc.fit_transform(data.Sales_Agent)
data.Location = enc.fit_transform(data.Location)
data.Delivery_Mode = enc.fit_transform(data.Delivery_Mode)
#data.day = enc.fit_transform(X.day)
data.Status = enc.fit_transform(data.Status)
#X = pd.get_dummies(X)
data.head(1)

,Created,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
0,2018-11-13 18:51:00,7,15,4,0,0,1


In [102]:
data.corr()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode,Status
Product_ID,1.000000,0.090025,0.017020,NaN,-0.057291,0.080662
Source,0.090025,1.000000,0.025951,NaN,-0.060237,-0.095889
Sales_Agent,0.017020,0.025951,1.000000,NaN,-0.152692,0.124987
Location,NaN,NaN,NaN,NaN,NaN,NaN
Delivery_Mode,-0.057291,-0.060237,-0.152692,NaN,1.000000,-0.031469
Status,0.080662,-0.095889,0.124987,NaN,-0.031469,1.000000


In [115]:
X = data.iloc[:,[1,2,3,4,5]]
y = data.Status
X.head()

,Product_ID,Source,Sales_Agent,Location,Delivery_Mode
0,7,15,4,0,0
4,14,2,8,0,0
9,14,2,8,0,0
11,7,0,8,0,2
12,7,2,5,0,0


In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)

In [118]:
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=80,max_depth=15,gamma=5,learning_rate=0.01)
model.fit(X_train,y_train)
y_predict = model.predict(X_test)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.709935897436
             precision    recall  f1-score   support

          0       0.74      0.40      0.52       245
          1       0.70      0.91      0.79       379

avg / total       0.72      0.71      0.69       624



col_0,0,1
Status,,
0,98,147
1,34,345


In [119]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_sample(X_train,y_train)

In [120]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=180,max_depth=20,gamma=8,learning_rate=0.05)
model.fit(X_train_smote,y_train_smote)

y_predict = model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

pd.crosstab(y_test,y_predict)

0.68108974359
             precision    recall  f1-score   support

          0       0.57      0.76      0.65       245
          1       0.80      0.63      0.71       379

avg / total       0.71      0.68      0.68       624



col_0,0,1
Status,,
0,187,58
1,141,238


In [72]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

parameters = {'kernel': ['rbf'], 'gamma': [0.1,1,5],
                     'C': [ 0.1,1,100, 1000,]}
                    
grid_model = RandomizedSearchCV(SVC(), parameters)
grid_model.fit(X_train_smote, y_train_smote)
print(grid_model.best_score_)
print(grid_model.best_params_)
y_predict = grid_model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.631430363864
{'kernel': 'rbf', 'gamma': 0.1, 'C': 1}
0.631939163498
             precision    recall  f1-score   support

          0       0.63      0.66      0.65       670
          1       0.63      0.60      0.62       645

avg / total       0.63      0.63      0.63      1315



col_0,0,1
Status,,
0,442,228
1,256,389


In [404]:
from sklearn.svm import SVC
model = SVC(kernel='rbf',gamma=5,C=10000)
model.fit(X_train_smote, y_train_smote)
y_predict = model.predict(X_test)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))

pd.crosstab(y_test,y_predict)

0.646706586826
             precision    recall  f1-score   support

          0       0.81      0.56      0.66      1132
          1       0.53      0.78      0.63       705

avg / total       0.70      0.65      0.65      1837



col_0,0,1
Status,,
0,637,495
1,154,551


In [405]:
parameters = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

parameters = {
 'max_depth': [200,300,None],
 'min_samples_leaf': [2,5,10],
 'n_estimators': [200,500,1000]}


In [406]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV

grid_model = RandomizedSearchCV(RandomForestClassifier(), parameters)
grid_model.fit(X_train_smote, y_train_smote)
print(grid_model.best_score_)
print(grid_model.best_params_)
y_predict = grid_model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.726462611391
{'n_estimators': 200, 'min_samples_leaf': 2, 'max_depth': 200}
0.715841045182


col_0,0,1
Status,,
0,846,286
1,236,469


In [108]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier

parameters = {
 'n_estimators': [80,100],
 'max_depth': [2,3,4],
 'gamma': [0.1,1],
 'learning_rate':[0.01,0.1,0.5],
 'colsample_bytree':[1],
 'subsample' : [1]}

grid_model = GridSearchCV(XGBClassifier(), parameters)
eval_set = [(X_train_smote, y_train_smote), (X_test.values, y_test.values)]
eval_metric = ["auc","error"]
%time grid_model.fit(X_train_smote,y_train_smote,eval_metric=eval_metric, eval_set=eval_set, verbose=True)

print(grid_model.best_score_)
print(grid_model.best_params_)
y_predict = grid_model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)



[0]	validation_0-auc:0.625097	validation_0-error:0.409449	validation_1-auc:0.621318	validation_1-error:0.483974
[1]	validation_0-auc:0.662682	validation_0-error:0.372891	validation_1-auc:0.673281	validation_1-error:0.363782
[2]	validation_0-auc:0.662682	validation_0-error:0.409449	validation_1-auc:0.673281	validation_1-error:0.483974
[3]	validation_0-auc:0.662682	validation_0-error:0.372891	validation_1-auc:0.673281	validation_1-error:0.363782
[4]	validation_0-auc:0.662682	validation_0-error:0.409449	validation_1-auc:0.673281	validation_1-error:0.483974
[5]	validation_0-auc:0.662682	validation_0-error:0.372891	validation_1-auc:0.673281	validation_1-error:0.363782
[6]	validation_0-auc:0.662682	validation_0-error:0.409449	validation_1-auc:0.673281	validation_1-error:0.483974
[7]	validation_0-auc:0.662682	validation_0-error:0.372891	validation_1-auc:0.673281	validation_1-error:0.363782
[8]	validation_0-auc:0.662682	validation_0-error:0.409449	validation_1-auc:0.673281	validation_1-error:0

[74]	validation_0-auc:0.675588	validation_0-error:0.37739	validation_1-auc:0.677341	validation_1-error:0.415064
[75]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[76]	validation_0-auc:0.675588	validation_0-error:0.37739	validation_1-auc:0.677341	validation_1-error:0.415064
[77]	validation_0-auc:0.675588	validation_0-error:0.37739	validation_1-auc:0.677341	validation_1-error:0.415064
[78]	validation_0-auc:0.675588	validation_0-error:0.37739	validation_1-auc:0.677341	validation_1-error:0.415064
[79]	validation_0-auc:0.675588	validation_0-error:0.37739	validation_1-auc:0.677341	validation_1-error:0.415064
[0]	validation_0-auc:0.644112	validation_0-error:0.38189	validation_1-auc:0.658446	validation_1-error:0.363782
[1]	validation_0-auc:0.644112	validation_0-error:0.38189	validation_1-auc:0.658446	validation_1-error:0.363782
[2]	validation_0-auc:0.644112	validation_0-error:0.38189	validation_1-auc:0.658446	validation_1-error:0.363

[68]	validation_0-auc:0.684419	validation_0-error:0.371766	validation_1-auc:0.686721	validation_1-error:0.411859
[69]	validation_0-auc:0.685059	validation_0-error:0.371766	validation_1-auc:0.686538	validation_1-error:0.411859
[70]	validation_0-auc:0.685384	validation_0-error:0.371766	validation_1-auc:0.686689	validation_1-error:0.411859
[71]	validation_0-auc:0.68595	validation_0-error:0.371766	validation_1-auc:0.687507	validation_1-error:0.411859
[72]	validation_0-auc:0.685482	validation_0-error:0.371766	validation_1-auc:0.686538	validation_1-error:0.411859
[73]	validation_0-auc:0.685846	validation_0-error:0.371766	validation_1-auc:0.686646	validation_1-error:0.411859
[74]	validation_0-auc:0.686037	validation_0-error:0.371766	validation_1-auc:0.686797	validation_1-error:0.411859
[75]	validation_0-auc:0.686106	validation_0-error:0.371766	validation_1-auc:0.686554	validation_1-error:0.411859
[76]	validation_0-auc:0.687522	validation_0-error:0.371766	validation_1-auc:0.686544	validation_1

[61]	validation_0-auc:0.672145	validation_0-error:0.38189	validation_1-auc:0.684007	validation_1-error:0.365385
[62]	validation_0-auc:0.672145	validation_0-error:0.38189	validation_1-auc:0.684007	validation_1-error:0.365385
[63]	validation_0-auc:0.672465	validation_0-error:0.38189	validation_1-auc:0.683878	validation_1-error:0.365385
[64]	validation_0-auc:0.680349	validation_0-error:0.38189	validation_1-auc:0.688046	validation_1-error:0.365385
[65]	validation_0-auc:0.680349	validation_0-error:0.38189	validation_1-auc:0.688057	validation_1-error:0.365385
[66]	validation_0-auc:0.680349	validation_0-error:0.368954	validation_1-auc:0.688057	validation_1-error:0.360577
[67]	validation_0-auc:0.680377	validation_0-error:0.368954	validation_1-auc:0.688434	validation_1-error:0.360577
[68]	validation_0-auc:0.680432	validation_0-error:0.368954	validation_1-auc:0.68852	validation_1-error:0.360577
[69]	validation_0-auc:0.680432	validation_0-error:0.368954	validation_1-auc:0.688509	validation_1-erro

[55]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[56]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[57]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[58]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[59]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[60]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[61]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[62]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[63]	validation_0-auc:0.67598	validation_0-error:0.37739	validation_1-auc:0.678041	validation_1-error:0.415064
[

...

[3]	validation_0-auc:0.740508	validation_0-error:0.32171	validation_1-auc:0.704938	validation_1-error:0.360577
[4]	validation_0-auc:0.772907	validation_0-error:0.300337	validation_1-auc:0.738824	validation_1-error:0.333333
[5]	validation_0-auc:0.77675	validation_0-error:0.297525	validation_1-auc:0.740633	validation_1-error:0.314103
[6]	validation_0-auc:0.777507	validation_0-error:0.295838	validation_1-auc:0.743999	validation_1-error:0.3125
[7]	validation_0-auc:0.778392	validation_0-error:0.298088	validation_1-auc:0.743428	validation_1-error:0.314103
[8]	validation_0-auc:0.781469	validation_0-error:0.291901	validation_1-auc:0.749922	validation_1-error:0.314103
[9]	validation_0-auc:0.781896	validation_0-error:0.292463	validation_1-auc:0.749836	validation_1-error:0.314103
[10]	validation_0-auc:0.785227	validation_0-error:0.289651	validation_1-auc:0.747741	validation_1-error:0.31891
[11]	validation_0-auc:0.78827	validation_0-error:0.287964	validation_1-auc:0.749577	validation_1-error:0.318

[76]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[77]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[78]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[79]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[80]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[81]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[82]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[83]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_1-error:0.307692
[84]	validation_0-auc:0.833434	validation_0-error:0.258718	validation_1-auc:0.747251	validation_

[49]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[50]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[51]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[52]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[53]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[54]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[55]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[56]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_1-error:0.322115
[57]	validation_0-auc:0.790183	validation_0-error:0.285714	validation_1-auc:0.753142	validation_

[22]	validation_0-auc:0.724074	validation_0-error:0.334083	validation_1-auc:0.718383	validation_1-error:0.358974
[23]	validation_0-auc:0.723871	validation_0-error:0.336333	validation_1-auc:0.718168	validation_1-error:0.365385
[24]	validation_0-auc:0.723378	validation_0-error:0.335208	validation_1-auc:0.716876	validation_1-error:0.365385
[25]	validation_0-auc:0.724349	validation_0-error:0.332958	validation_1-auc:0.716919	validation_1-error:0.365385
[26]	validation_0-auc:0.7272	validation_0-error:0.332396	validation_1-auc:0.721571	validation_1-error:0.365385
[27]	validation_0-auc:0.728179	validation_0-error:0.330146	validation_1-auc:0.7217	validation_1-error:0.360577
[28]	validation_0-auc:0.727191	validation_0-error:0.329021	validation_1-auc:0.721054	validation_1-error:0.360577
[29]	validation_0-auc:0.726524	validation_0-error:0.330146	validation_1-auc:0.720968	validation_1-error:0.360577
[30]	validation_0-auc:0.727661	validation_0-error:0.332396	validation_1-auc:0.721081	validation_1-er

col_0,0,1
Status,,
0,176,69
1,139,240


In [442]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from xgboost import XGBClassifier
model = XGBClassifier(n_estimators=250,max_depth=2,gamma=3,learning_rate=0.001)
model.fit(X_train_smote,y_train_smote)
y_predict = model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.624387588459
             precision    recall  f1-score   support

          0       0.89      0.44      0.59      1135
          1       0.50      0.91      0.65       702

avg / total       0.75      0.62      0.62      1837



col_0,0,1
Status,,
0,505,630
1,60,642


In [441]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=300)
model.fit(X_train_smote,y_train_smote)
y_predict = model.predict(X_test.values)
print(accuracy_score(y_test,y_predict))
print(classification_report(y_test,y_predict))
pd.crosstab(y_test,y_predict)

0.68426782798
             precision    recall  f1-score   support

          0       0.79      0.67      0.72      1135
          1       0.57      0.71      0.63       702

avg / total       0.71      0.68      0.69      1837



col_0,0,1
Status,,
0,758,377
1,203,499
